<a href="https://colab.research.google.com/github/ItsShi/GraphNeuralNetworkforSimilarityLearning/blob/main/QM7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Matching Networks for the Similarity Learning

## Some dependencies and imports

These are all the dependencies that will be used in this notebook.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 30 10:05:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:

!pip uninstall torch  #1.10.0+cu111 会报错
!pip install torch==1.5 #1.3或者1.2
!nvcc --version
print("\n")

import torch
import torch.nn as nn
print(torch.__version__)

# !pip install torch-scatter 
# !pip install torch-sparse 
# !pip install torch-geometric
# import torch_geometric

Found existing installation: torch 1.10.0+cu111
Uninstalling torch-1.10.0+cu111:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.10.0+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.10.0+cu111
     |████████████████████████████████| 752.0 MB 9.2 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.0 which is incompatible.
nvcc: NVIDIA (R) Cuda compiler driv

In [ ]:
#dgl
# !git init
# !git clone https://github.com/dmlc/dgl.git
# !git submodule init
# !git submodule update

!pip install dgl-cu101

     |████████████████████████████████| 36.2 MB 7.7 MB/s 


In [ ]:
# #gpu
# if torch.cuda.is_available():
#   !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
#   !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html #gpu 
#   !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
#   import torch_geometric

# else:
#   !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cpu.html
#   !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cpu.html  #cpu pytorch1.10.0
#   !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
#   import torch_geometric

In [ ]:
#pascal VOC dataset
# !pip install opencv-python
# !pip3 install torchvision

# print("下载github/..zip：")
# !pip install https://github.com/fastai/fastai/archive/master.zip

# print("更新和安装: -h（帮助），-y（当安装过程提示选择全部为yes），-q（不显示安装的过程）")
# !apt update && apt install -y libsm6 libxext6

# print("下载pytorch/..whl: ")
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

# !mkdir data  

# #Wget： supports downloading via HTTP, HTTPS, and FTP
# !wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P data/ 

# print("\n")
# !wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P data/

# #tar(tarball): for collecting many files into one archive file, "tape archive"
# print("归档.tar：")
# !tar -xf data/VOCtrainval_06-Nov-2007.tar -C data/

# print("解压zip：")
# !unzip data/PASCAL_VOC.zip -d data/

# print("remove文件：")
# !rm -rf data/PASCAL_VOC.zip data/VOCtrainval_06-Nov-2007.tar

In [ ]:
#compare existing deep graph matching algorithms under different datasets
#provides a unified data interface and an evaluating platform 
#pygmtools supports 5 datasets, including PascalVOC, Willow-Object, SPair-71k, CUB2011 and IMC-PT-SparseGM.
# !pip uninstall Pillow
# !pip install Pillow==7.2.0
# !pip install pygmtools
#dataset.py: to download dataset and process the dataset into a json file, and also save train set and test set.
#benchmark.py:  to fetch data from json file and evaluate prediction result.
#dataset_config.py: Fixed dataset settings, mostly dataset path and classes.

'''
evaluation metrics include matching_precision (p), matching_recall (r) and f1_score (f1). 
Also, to measure the reliability of the evaluation result, 
we define coverage (cvg) for each class in the dataset as number of evaluated pairs in the class / number of all possible pairs in the class. 
Therefore, larger coverage refers to higher reliability.
'''

'''
#example
from pygmtools.benchmark import Benchmark

# Define Benchmark on PascalVOC.
bm = Benchmark(name='PascalVOC', sets='train', 
               obj_resize=(256, 256), problem='2GM',
               filter='intersection')

# Random fetch data and ground truth.
data_list, gt_dict, _ = bm.rand_get_data(cls=None, num=2)
'''

"\n#example\nfrom pygmtools.benchmark import Benchmark\n\n# Define Benchmark on PascalVOC.\nbm = Benchmark(name='PascalVOC', sets='train', \n               obj_resize=(256, 256), problem='2GM',\n               filter='intersection')\n\n# Random fetch data and ground truth.\ndata_list, gt_dict, _ = bm.rand_get_data(cls=None, num=2)\n"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
'''
import matplotlib
import networkx
import numpy
import pandas
# import scikitlearn
import scipy
import texttable
import tqdm
import Cython

print(matplotlib. __version__)
print(networkx. __version__)
print(numpy. __version__)
print(pandas. __version__)
print(matplotlib. __version__)
print(scipy. __version__)
print(texttable. __version__)
print(tqdm. __version__)
print(Cython. __version__)

# matplotlib==3.3.4
# networkx==2.5
# numpy==1.20.1
# pandas==1.2.3
# scikit-learn==0.24.1
# scipy==1.6.1
# texttable==1.6.3
# tqdm==4.59.0
# Cython==0.29.23

'''

'\nimport matplotlib\nimport networkx\nimport numpy\nimport pandas\n# import scikitlearn\nimport scipy\nimport texttable\nimport tqdm\nimport Cython\n\nprint(matplotlib. __version__)\nprint(networkx. __version__)\nprint(numpy. __version__)\nprint(pandas. __version__)\nprint(matplotlib. __version__)\nprint(scipy. __version__)\nprint(texttable. __version__)\nprint(tqdm. __version__)\nprint(Cython. __version__)\n\n# matplotlib==3.3.4\n# networkx==2.5\n# numpy==1.20.1\n# pandas==1.2.3\n# scikit-learn==0.24.1\n# scipy==1.6.1\n# texttable==1.6.3\n# tqdm==4.59.0\n# Cython==0.29.23\n\n'

In [ ]:
#%loadpy '-/Users/xiaoweishi/Downloads/Graph-Matching-Networks-main/segment.py'

交叉图注意力的匹配机制对输入的一对图进行联合推理，计算它们之间的相似度得分

#Background：
现有的GNN通过迭代聚合局部结构信息的传播过程设计和计算图节点表示，对图元素的排列保持不变，然后将这些节点表示直接用于节点分类，或汇集到图向量中进行图分类。
GNN 对监督分类或回归之外的问题的研究相对较少。

#主要任务：
1 证明GNN可以为相似性学习提供嵌入  2 设计GMN计算图间相似匹配

相关工作：WL研究图同构（相同）与本文要研究图相似有区别，图核更重于设计，基于图论，没有基于学习，不端到端，Siamese网络可通过网格化计算相似度

#标准GNN和本文的GMN：
GEM：编码器-传播层-聚合（MLP构造节点和边的嵌入向量h和e-MLP或RNN将节点嵌入映射到新的节点表示 多层传播后每个节点的表示其局部邻域中积累信息-输入一系列节点表示计算出一个图表示 带有门控权重的聚合方法可以滤波掉无关信息，比求和等方法好，有了两个图的表示就可以在欧式空间计算相似度了，比如欧氏距离，余弦相似度，汉明相似度等如果没有传播层就是计算独立点的表示，再池化到整个图上，但这样就忽视了结构信息，单纯在处理一堆独立数据）
GMN：不单独嵌入每个图表示，联合计算相似度，改变每个传播层的节点更新模块，不仅考虑边上的聚合信息，而且考虑图间匹配向量，用来衡量一个图中的节点与另一个图中的一个或多个节点的匹配程度
GMN花时间，注意力模块有两个好处：两图完美匹配时，注意力权重达到峰值，μ的和为0，这样两图在下一轮传播中会计算相同的表示图间差异会记录在μ的和中，通过传播逐渐放大，可以让匹配模型对差异更加敏感。
匹配模型可以基于其他图来改变当前图的表示，也即调整图的表示使得图间差异愈发明显便于匹配

#训练学习：
可以选择成对训练，还是三元训练
成对训练需要标签positive (相似) or negative (不相似)
而三元训练只需要相对相似，即 G1 更接近 G2 还是 G3

使用欧式相似度，基于边缘的成对损失

$$
L_\mathrm{pair} = \mathbb{E}_{(G_1, G_2, t)}[\max\{0, \gamma - t(1 - d(G_1, G_2))\}]
$$


其中t ∈ {−1, 1}作为这一对图的标签，γ > 0是边缘参数，d是欧氏距离
当这一对相似（t=1）时，损失L收敛到0会让d< 1−γ；当 t = −1 时，d> 1+γ。

给定 G1 和 G2 比 G1 和 G3 更接近的三元组，优化以下基于边距的三元组损失：




损失收敛到0可以通过边缘γ让d(G1, G2)比d(G1, G3) 小

将图表示成二进制-1或1，可以最小化相似对的汉明距离，最大化不相似对的距离（扩大类间距，缩小类内距的方法吗？）这样可以提高检索效率
于是可以引入tanh变化优化损失函数

$$
s(G_1, G_2) = \frac{1}{H}\sum_{i=1}^H \tanh(h_{G_1, i}) \cdot \tanh(h_{G_2, i}),
$$

$$
L_\mathrm{pair} = \mathbb{E}_{(G_1, G_2, t)}[(t - s(G_1, G_2))^2] / 4.
$$

其中
是近似的平均汉明距离（有点像多图递归矩阵补全代码里的那个损失函数处理）这样会比基于边缘的损失更稳定



#实验 训练过程：：
GED被定义为图间最小距离，因为是NP难所以近似（但是既然近似计算相似度的这些算法里没有用到GED或者其思想，那为啥还要说它呢，不就和它没关系吗？）


随机构造一个n节点，边概率p的图G1，然后再随机替换kp条边为新边构造相似样本G2，替换kn条边构造不相似样本G3，kp<kn（原来如此，代码里构造数据集的过程类似于GED，这不是NP难吗，怪不得运行不出来？）模型需要预测相似对（G1，G2）的相似度高于不相似对（G1，G3）

对于 GMN，可以将图间注意力可视化，以进一步了解它是如何工作的。当两个图匹配时，注意力权重可以很好地对齐节点，而当它们不匹配时，往往会关注度数较高的节点。 然而，该模式不像标准注意力模型那样具有可解释性。


从相似性学习设置中学习的表示也可以轻松地泛化到训练期间未见过的类的数据（零样本泛化）。

### Graph embedding model


#### The graph encoder


$$d(G_1, G_2) = d_H(embed(G_1), embed(G_2)),$$


1. $embed$ maps any graph  $G$  into an $H$ -dimensional vector:

$embed(G_1) = $

1) mapping: 

$$\begin{array}{rcl}
h_i^{(0)} &=& \mathrm{MLP_{node}}(x_i) \\
e_{ij} &=& \mathrm{MLP_{edge}}(x_{ij})
\end{array}
$$

2) iterative message passing: 

$$\begin{array}{rcl}
m_{i\rightarrow j} &=& f_\mathrm{message}(h_i^{(t)}, h_j^{(t)}, e_{ij}) \\
h_i^{(t+1)} &=& f_\mathrm{node}(h_i^{(t)}, \sum_{j:(j,i)\in E} m_{j\rightarrow i})
\end{array}
$$

3) aggregate node representations to get graph representations


$$h_G = \mathrm{MLP_G}\left(\sum_{i\in V} h_i^{(T)}\right).$$
$$h_G = \mathrm{MLP_G}\left(\sum_{i\in V} \sigma(\mathrm{MLP_{gate}}(h_i^{(T)})) \odot \mathrm{MLP}(h_i^{(T)})\right).$$

2. $d_H$ is an existing distance metric\:

Euclidean distance: $d_H(x, y) = \sqrt{\sum_{i=1}^H (x_i - y_i)^2}$,

or Hamming distance: $d_H(x, y)=\sum_{i=1}^H \mathbb{I}[x_i \ne y_i]$

  

 






节点嵌入：

FCCN 层，（Linear）. ReLU 作为激活函数。

聚合层：

聚合函数（GRU，FCCN）

Adam优化器。

In [ ]:
import torch.nn as nn

class GraphEncoder(nn.Module):
    """Encoder module that projects node and edge features to some embeddings."""

    def __init__(self,
                 node_feature_dim,
                 edge_feature_dim,
                 node_hidden_sizes=None,
                 edge_hidden_sizes=None,
                 name='graph-encoder'):
      
        super(GraphEncoder, self).__init__()

        self._node_feature_dim = node_feature_dim
        self._edge_feature_dim = edge_feature_dim
        self._node_hidden_sizes = node_hidden_sizes if node_hidden_sizes else None
        self._edge_hidden_sizes = edge_hidden_sizes
        self._build_model()

    def _build_model(self):
        layer = []
        layer.append(nn.Linear(self._node_feature_dim, self._node_hidden_sizes[0]))
        for i in range(1, len(self._node_hidden_sizes)):
            layer.append(nn.ReLU())
            layer.append(nn.Linear(self._node_hidden_sizes[i - 1], self._node_hidden_sizes[i]))
        self.MLP1 = nn.Sequential(*layer)

        if self._edge_hidden_sizes is not None:
            layer = []
            layer.append(nn.Linear(self._edge_feature_dim, self._edge_hidden_sizes[0]))
            for i in range(1, len(self._edge_hidden_sizes)):
                layer.append(nn.ReLU())
                layer.append(nn.Linear(self._edge_hidden_sizes[i - 1], self._edge_hidden_sizes[i]))
            self.MLP2 = nn.Sequential(*layer)
        else:
            self.MLP2 = None

    def forward(self, node_features, edge_features=None):
        """Encode node and edge features.
        Args:
          node_features: [n_nodes, node_feat_dim] float tensor.
          edge_features: if provided, should be [n_edges, edge_feat_dim] float
            tensor.
        Returns:
          node_outputs: [n_nodes, node_embedding_dim] float tensor, node embeddings.
          edge_outputs: if edge_features is not None and edge_hidden_sizes is not
            None, this is [n_edges, edge_embedding_dim] float tensor, edge
            embeddings; otherwise just the input edge_features.
        """
        if self._node_hidden_sizes is None:
            node_outputs = node_features
        else:
            node_outputs = self.MLP1(node_features)
        if edge_features is None or self._edge_hidden_sizes is None:
            edge_outputs = edge_features
        else:
            edge_outputs = self.MLP2(edge_features)

        return node_outputs, edge_outputs

#### The message passing layers

In [ ]:
def unsorted_segment_sum(data, segment_ids, num_segments):
    """
    Computes the sum along segments of a tensor. Analogous to tf.unsorted_segment_sum.
    :param data: A tensor whose segments are to be summed.
    :param segment_ids: The segment indices tensor.
    :param num_segments: The number of segments.
    :return: A tensor of same data type as the data argument.
    """
    # --------------print("\ndata: {}\nsegment_ids: {}\nnum_segments: {}".format(data, segment_ids, num_segments))
    assert all([i in data.shape for i in segment_ids.shape]), "segment_ids.shape should be a prefix of data.shape"

    # Encourage to use the below code when a deterministic result is
    # needed (reproducibility). However, the code below is with low efficiency.

    # tensor = torch.zeros(num_segments, data.shape[1]).cuda()
    # for index in range(num_segments):
    #     tensor[index, :] = torch.sum(data[segment_ids == index, :], dim=0)
    # return tensor

    #----------------print("\ndata.shape:{} and segment_ids.shape: {}".format(data.shape, segment_ids.shape))
    if len(segment_ids.shape) == 1:
      if torch.cuda.is_available():
        s = torch.prod(torch.tensor(data.shape[1:])).long().cuda()
        segment_ids = segment_ids.repeat_interleave(s).view(segment_ids.shape[0], *data.shape[1:])
      else: 
        s = torch.prod(torch.tensor(data.shape[1:])).long()
        segment_ids = segment_ids.repeat_interleave(s).view(segment_ids.shape[0], *data.shape[1:])
    #----------------- print("\ndata.shape:{} and segment_ids.shape: {}".format(data.shape, segment_ids.shape))
    assert data.shape == segment_ids.shape, "data.shape and segment_ids.shape should be equal"

    shape = [num_segments] + list(data.shape[1:])
    if torch.cuda.is_available():
      tensor = torch.zeros(*shape).cuda().scatter_add(0, segment_ids, data) 
    else:
      tensor = torch.zeros(*shape).scatter_add(0, segment_ids, data)

    tensor = tensor.type(data.dtype)
    return tensor

In [ ]:

def graph_prop_once(node_states,
                    from_idx,
                    to_idx,
                    message_net,
                    aggregation_module=None,
                    edge_features=None):
    """One round of propagation (message passing) in a graph.
    Args:
      node_states: [n_nodes, node_state_dim] float tensor, node state vectors, one
        row for each node.
      from_idx: [n_edges] int tensor, index of the from nodes.
      to_idx: [n_edges] int tensor, index of the to nodes.
      message_net: a network that maps concatenated edge inputs to message
        vectors.
      aggregation_module: a module that aggregates messages on edges to aggregated
        messages for each node.  Should be a callable and can be called like the
        following,
        `aggregated_messages = aggregation_module(messages, to_idx, n_nodes)`,
        where messages is [n_edges, edge_message_dim] tensor, to_idx is the index
        of the to nodes, i.e. where each message should go to, and n_nodes is an
        int which is the number of nodes to aggregate into.
      edge_features: if provided, should be a [n_edges, edge_feature_dim] float
        tensor, extra features for each edge.
    Returns:
      aggregated_messages: an [n_nodes, edge_message_dim] float tensor, the
        aggregated messages, one row for each node.
    """
    from_states = node_states[from_idx]
    to_states = node_states[to_idx]
    edge_inputs = [from_states, to_states]

    if edge_features is not None:
        edge_inputs.append(edge_features)

    edge_inputs = torch.cat(edge_inputs, dim=-1)
    messages = message_net(edge_inputs)
    #-----------------------print("messages: {}\nto_idx: {}\n".format(messages, to_idx))
    tensor = unsorted_segment_sum(messages, to_idx, node_states.shape[0])
    return tensor



In [ ]:
class GraphPropLayer(nn.Module):
    """Implementation of a graph propagation (message passing) layer."""

    def __init__(self,
                 node_state_dim,
                 edge_state_dim,
                 edge_hidden_sizes,  # int
                 node_hidden_sizes,  # int
                 edge_net_init_scale=0.1,
                 node_update_type='residual',
                 use_reverse_direction=True,
                 reverse_dir_param_different=True,
                 layer_norm=False,
                 prop_type='embedding',
                 name='graph-net'):
        """Constructor.
        Args:
          node_state_dim: int, dimensionality of node states.
          edge_hidden_sizes: list of ints, hidden sizes for the edge message
            net, the last element in the list is the size of the message vectors.
          node_hidden_sizes: list of ints, hidden sizes for the node update
            net.
          edge_net_init_scale: initialization scale for the edge networks.  This
            is typically set to a small value such that the gradient does not blow
            up.
          node_update_type: type of node updates, one of {mlp, gru, residual}.
          use_reverse_direction: set to True to also propagate messages in the
            reverse direction.
          reverse_dir_param_different: set to True to have the messages computed
            using a different set of parameters than for the forward direction.
          layer_norm: set to True to use layer normalization in a few places.
          name: name of this module.
        """
        super(GraphPropLayer, self).__init__()

        self._node_state_dim = node_state_dim
        self._edge_state_dim = edge_state_dim
        self._edge_hidden_sizes = edge_hidden_sizes[:]

        # output size is node_state_dim
        self._node_hidden_sizes = node_hidden_sizes[:] + [node_state_dim]
        self._edge_net_init_scale = edge_net_init_scale
        self._node_update_type = node_update_type

        self._use_reverse_direction = use_reverse_direction
        self._reverse_dir_param_different = reverse_dir_param_different

        self._layer_norm = layer_norm
        self._prop_type = prop_type
        self.build_model()

        if self._layer_norm:
            self.layer_norm1 = nn.LayerNorm()
            self.layer_norm2 = nn.LayerNorm()

    def build_model(self):
        layer = []
        layer.append(nn.Linear(self._node_state_dim*2 + self._edge_state_dim, self._edge_hidden_sizes[0]))
        for i in range(1, len(self._edge_hidden_sizes)):
            layer.append(nn.ReLU())
            layer.append(nn.Linear(self._edge_hidden_sizes[i - 1], self._edge_hidden_sizes[i]))
        self._message_net = nn.Sequential(*layer)

        # optionally compute message vectors in the reverse direction
        if self._use_reverse_direction:
            if self._reverse_dir_param_different:
                layer = []
                layer.append(nn.Linear(self._node_state_dim*2 + self._edge_state_dim, self._edge_hidden_sizes[0]))
                for i in range(1, len(self._edge_hidden_sizes)):
                    layer.append(nn.ReLU())
                    layer.append(nn.Linear(self._edge_hidden_sizes[i - 1], self._edge_hidden_sizes[i]))
                self._reverse_message_net = nn.Sequential(*layer)
            else:
                self._reverse_message_net = self._message_net

        if self._node_update_type == 'gru':
            if self._prop_type == 'embedding':
                self.GRU = torch.nn.GRU(self._node_state_dim * 2, self._node_state_dim)
            elif self._prop_type == 'matching':
                self.GRU = torch.nn.GRU(self._node_state_dim * 3, self._node_state_dim)
        else:
            layer = []
            if self._prop_type == 'embedding':
                layer.append(nn.Linear(self._node_state_dim * 3, self._node_hidden_sizes[0]))
            elif self._prop_type == 'matching':
                layer.append(nn.Linear(self._node_state_dim * 4, self._node_hidden_sizes[0]))
            for i in range(1, len(self._node_hidden_sizes)):
                layer.append(nn.ReLU())
                layer.append(nn.Linear(self._node_hidden_sizes[i - 1], self._node_hidden_sizes[i]))
            self.MLP = nn.Sequential(*layer)

    def _compute_aggregated_messages(
            self, node_states, from_idx, to_idx, edge_features=None):
        """Compute aggregated messages for each node.
        Args:
          node_states: [n_nodes, input_node_state_dim] float tensor, node states.
          from_idx: [n_edges] int tensor, from node indices for each edge.
          to_idx: [n_edges] int tensor, to node indices for each edge.
          edge_features: if not None, should be [n_edges, edge_embedding_dim]
            tensor, edge features.
        Returns:
          aggregated_messages: [n_nodes, aggregated_message_dim] float tensor, the
            aggregated messages for each node.
        """

        aggregated_messages = graph_prop_once(
            node_states,
            from_idx,
            to_idx,
            self._message_net,
            aggregation_module=None,
            edge_features=edge_features)

        # optionally compute message vectors in the reverse direction
        if self._use_reverse_direction:
            reverse_aggregated_messages = graph_prop_once(
                node_states,
                to_idx,
                from_idx,
                self._reverse_message_net,
                aggregation_module=None,
                edge_features=edge_features)

            aggregated_messages += reverse_aggregated_messages

        if self._layer_norm:
            aggregated_messages = self.layer_norm1(aggregated_messages)

        return aggregated_messages

    def _compute_node_update(self,
                             node_states,
                             node_state_inputs,
                             node_features=None):
        """Compute node updates.
        Args:
          node_states: [n_nodes, node_state_dim] float tensor, the input node
            states.
          node_state_inputs: a list of tensors used to compute node updates.  Each
            element tensor should have shape [n_nodes, feat_dim], where feat_dim can
            be different.  These tensors will be concatenated along the feature
            dimension.
          node_features: extra node features if provided, should be of size
            [n_nodes, extra_node_feat_dim] float tensor, can be used to implement
            different types of skip connections.
        Returns:
          new_node_states: [n_nodes, node_state_dim] float tensor, the new node
            state tensor.
        Raises:
          ValueError: if node update type is not supported.
        """
        if self._node_update_type in ('mlp', 'residual'):
            node_state_inputs.append(node_states)
        if node_features is not None:
            node_state_inputs.append(node_features)

        if len(node_state_inputs) == 1:
            node_state_inputs = node_state_inputs[0]
        else:
            node_state_inputs = torch.cat(node_state_inputs, dim=-1)

        if self._node_update_type == 'gru':
            node_state_inputs = torch.unsqueeze(node_state_inputs, 0)
            node_states = torch.unsqueeze(node_states, 0)
            _, new_node_states = self.GRU(node_state_inputs, node_states)
            new_node_states = torch.squeeze(new_node_states)
            return new_node_states
        else:
            mlp_output = self.MLP(node_state_inputs)
            if self._layer_norm:
                mlp_output = nn.self.layer_norm2(mlp_output)
            if self._node_update_type == 'mlp':
                return mlp_output
            elif self._node_update_type == 'residual':
                return node_states + mlp_output
            else:
                raise ValueError('Unknown node update type %s' % self._node_update_type)

    def forward(self,
                node_states,
                from_idx,
                to_idx,
                edge_features=None,
                node_features=None):
        """Run one propagation step.
        Args:
          node_states: [n_nodes, input_node_state_dim] float tensor, node states.
          from_idx: [n_edges] int tensor, from node indices for each edge.
          to_idx: [n_edges] int tensor, to node indices for each edge.
          edge_features: if not None, should be [n_edges, edge_embedding_dim]
            tensor, edge features.
          node_features: extra node features if provided, should be of size
            [n_nodes, extra_node_feat_dim] float tensor, can be used to implement
            different types of skip connections.
        Returns:
          node_states: [n_nodes, node_state_dim] float tensor, new node states.
        """
        aggregated_messages = self._compute_aggregated_messages(
            node_states, from_idx, to_idx, edge_features=edge_features)

        return self._compute_node_update(node_states,
                                         [aggregated_messages],
                                         node_features=node_features)

#### Graph aggregator

In [ ]:
class GraphAggregator(nn.Module):
    """This module computes graph representations by aggregating from parts."""

    def __init__(self,
                 node_hidden_sizes,
                 graph_transform_sizes=None,
                 input_size=None,
                 gated=True,
                 aggregation_type='sum',
                 name='graph-aggregator'):
        """Constructor.
        Args:
          node_hidden_sizes: the hidden layer sizes of the node transformation nets.
            The last element is the size of the aggregated graph representation.
          graph_transform_sizes: sizes of the transformation layers on top of the
            graph representations.  The last element of this list is the final
            dimensionality of the output graph representations.
          gated: set to True to do gated aggregation, False not to.
          aggregation_type: one of {sum, max, mean, sqrt_n}.
          name: name of this module.
        """
        super(GraphAggregator, self).__init__()

        self._node_hidden_sizes = node_hidden_sizes
        self._graph_transform_sizes = graph_transform_sizes
        self._graph_state_dim = node_hidden_sizes[-1]
        self._input_size = input_size
        #  The last element is the size of the aggregated graph representation.
        self._gated = gated
        self._aggregation_type = aggregation_type
        self._aggregation_op = None
        self.MLP1, self.MLP2 = self.build_model()

    def build_model(self):
        node_hidden_sizes = self._node_hidden_sizes
        if self._gated:
            node_hidden_sizes[-1] = self._graph_state_dim * 2

        layer = []
        layer.append(nn.Linear(self._input_size[0], node_hidden_sizes[0]))
        for i in range(1, len(node_hidden_sizes)):
            layer.append(nn.ReLU())
            layer.append(nn.Linear(node_hidden_sizes[i - 1], node_hidden_sizes[i]))
        MLP1 = nn.Sequential(*layer)

        if (self._graph_transform_sizes is not None and
                len(self._graph_transform_sizes) > 0):
            layer = []
            layer.append(nn.Linear(self._graph_state_dim, self._graph_transform_sizes[0]))
            for i in range(1, len(self._graph_transform_sizes)):
                layer.append(nn.ReLU())
                layer.append(nn.Linear(self._graph_transform_sizes[i - 1], self._graph_transform_sizes[i]))
            MLP2 = nn.Sequential(*layer)

        return MLP1, MLP2

    def forward(self, node_states, graph_idx, n_graphs):
        """Compute aggregated graph representations.
        Args:
          node_states: [n_nodes, node_state_dim] float tensor, node states of a
            batch of graphs concatenated together along the first dimension.
          graph_idx: [n_nodes] int tensor, graph ID for each node.
          n_graphs: integer, number of graphs in this batch.
        Returns:
          graph_states: [n_graphs, graph_state_dim] float tensor, graph
            representations, one row for each graph.
        """

        node_states_g = self.MLP1(node_states)

        if self._gated:
            gates = torch.sigmoid(node_states_g[:, :self._graph_state_dim])
            node_states_g = node_states_g[:, self._graph_state_dim:] * gates

        graph_states = unsorted_segment_sum(node_states_g, graph_idx, n_graphs)

        if self._aggregation_type == 'max':
            # reset everything that's smaller than -1e5 to 0.
            graph_states *= torch.FloatTensor(graph_states > -1e5)
        # transform the reduced graph states further


        if (self._graph_transform_sizes is not None and
                len(self._graph_transform_sizes) > 0):
            graph_states = self.MLP2(graph_states)

        return graph_states

#### Putting them together

In [ ]:
class GraphEmbeddingNet(nn.Module):
    """A graph to embedding mapping network."""

    def __init__(self,
                 encoder,
                 aggregator,
                 node_state_dim,
                 edge_state_dim,
                 edge_hidden_sizes,
                 node_hidden_sizes,
                 n_prop_layers,
                 share_prop_params=False,
                 edge_net_init_scale=0.1,
                 node_update_type='residual',
                 use_reverse_direction=True,
                 reverse_dir_param_different=True,
                 layer_norm=False,
                 layer_class=GraphPropLayer,
                 prop_type='embedding',
                 name='graph-embedding-net'):
        """Constructor.
        Args:
          encoder: GraphEncoder, encoder that maps features to embeddings.
          aggregator: GraphAggregator, aggregator that produces graph
            representations.
          node_state_dim: dimensionality of node states.
          edge_hidden_sizes: sizes of the hidden layers of the edge message nets.
          node_hidden_sizes: sizes of the hidden layers of the node update nets.
          n_prop_layers: number of graph propagation layers.
          share_prop_params: set to True to share propagation parameters across all
            graph propagation layers, False not to.
          edge_net_init_scale: scale of initialization for the edge message nets.
          node_update_type: type of node updates, one of {mlp, gru, residual}.
          use_reverse_direction: set to True to also propagate messages in the
            reverse direction.
          reverse_dir_param_different: set to True to have the messages computed
            using a different set of parameters than for the forward direction.
          layer_norm: set to True to use layer normalization in a few places.
          name: name of this module.
        """
        super(GraphEmbeddingNet, self).__init__()

        self._encoder = encoder
        self._aggregator = aggregator
        self._node_state_dim = node_state_dim
        self._edge_state_dim = edge_state_dim
        self._edge_hidden_sizes = edge_hidden_sizes
        self._node_hidden_sizes = node_hidden_sizes
        self._n_prop_layers = n_prop_layers
        self._share_prop_params = share_prop_params
        self._edge_net_init_scale = edge_net_init_scale
        self._node_update_type = node_update_type
        self._use_reverse_direction = use_reverse_direction
        self._reverse_dir_param_different = reverse_dir_param_different
        self._layer_norm = layer_norm
        self._prop_layers = []
        self._prop_layers = nn.ModuleList()
        self._layer_class = layer_class
        self._prop_type = prop_type
        self.build_model()

    def _build_layer(self, layer_id):
        """Build one layer in the network."""
        return self._layer_class(
            self._node_state_dim,
            self._edge_state_dim,
            self._edge_hidden_sizes,
            self._node_hidden_sizes,
            edge_net_init_scale=self._edge_net_init_scale,
            node_update_type=self._node_update_type,
            use_reverse_direction=self._use_reverse_direction,
            reverse_dir_param_different=self._reverse_dir_param_different,
            layer_norm=self._layer_norm,
            prop_type=self._prop_type)
        # name='graph-prop-%d' % layer_id)

    def _apply_layer(self,
                     layer,
                     node_states,
                     from_idx,
                     to_idx,
                     graph_idx,
                     n_graphs,
                     edge_features):
        """Apply one layer on the given inputs."""
        del graph_idx, n_graphs
        return layer(node_states, from_idx, to_idx, edge_features=edge_features)

    def build_model(self):
        if len(self._prop_layers) < self._n_prop_layers:
            # build the layers
            for i in range(self._n_prop_layers):
                if i == 0 or not self._share_prop_params:
                    layer = self._build_layer(i)
                else:
                    layer = self._prop_layers[0]
                self._prop_layers.append(layer)

    def forward(self,
                node_features,
                edge_features,
                from_idx,
                to_idx,
                graph_idx,
                n_graphs):
        """Compute graph representations.
        Args:
          node_features: [n_nodes, node_feat_dim] float tensor.
          edge_features: [n_edges, edge_feat_dim] float tensor.
          from_idx: [n_edges] int tensor, index of the from node for each edge.
          to_idx: [n_edges] int tensor, index of the to node for each edge.
          graph_idx: [n_nodes] int tensor, graph id for each node.
          n_graphs: int, number of graphs in the batch.
        Returns:
          graph_representations: [n_graphs, graph_representation_dim] float tensor,
            graph representations.
        """

        node_features, edge_features = self._encoder(node_features, edge_features)
        node_states = node_features

        layer_outputs = [node_states]

        for layer in self._prop_layers:
            # node_features could be wired in here as well, leaving it out for now as
            # it is already in the inputs
            node_states = self._apply_layer(
                layer,
                node_states,
                from_idx,
                to_idx,
                graph_idx,
                n_graphs,
                edge_features)
            layer_outputs.append(node_states)

        # these tensors may be used e.g. for visualization
        self._layer_outputs = layer_outputs
        return self._aggregator(node_states, graph_idx, n_graphs)

    def reset_n_prop_layers(self, n_prop_layers):
        """Set n_prop_layers to the provided new value.
        This allows us to train with certain number of propagation layers and
        evaluate with a different number of propagation layers.
        This only works if n_prop_layers is smaller than the number used for
        training, or when share_prop_params is set to True, in which case this can
        be arbitrarily large.
        Args:
          n_prop_layers: the new number of propagation layers to set.
        """
        self._n_prop_layers = n_prop_layers

    @property
    def n_prop_layers(self):
        return self._n_prop_layers

    def get_layer_outputs(self):
        """Get the outputs at each layer."""
        if hasattr(self, '_layer_outputs'):
            return self._layer_outputs
        else:
            raise ValueError('No layer outputs available.')


### Graph matching networks

#### A few similarity functions

$$d(G_1, G_2) = d_H(embed\_and\_match(G_1, G_2))$$

1. $embed\_and\_match(G_1, G_2)$ =

1) mapping: 

$$\begin{array}{rcl}
h_i^{(0)} &=& \mathrm{MLP_{node}}(x_i) \\
e_{ij} &=& \mathrm{MLP_{edge}}(x_{ij})
\end{array}
$$

2) passing cross-graph messages: 
$$\begin{array}{rcl}
m_{i\rightarrow j} &=& f_\mathrm{message}(h_i^{(t)}, h_j^{(t)}, e_{ij}) \\
\end{array}
$$
cross graph attention weight:
$$\begin{array}{rcl}
a_{i\rightarrow j} &=& \frac{\exp(s(h_i^{(t)}, h_j^{(t)}))}{\sum_j \exp(s(h_i^{(t)}, h_j^{(t)}))} , GAT: a_{i\rightarrow j} &=& \frac{\exp(LeakyReLU([Wh_i||Wh_j]))}{\sum_j \exp(LeakyReLU([Wh_i||Wh_j]))}\\
a_{j\rightarrow i} &=& \frac{\exp(s(h_i^{(t)}, h_j^{(t)}))}{\sum_i \exp(s(h_i^{(t)}, h_j^{(t)}))}
\end{array}
$$
take the difference of attention-weighted sum of node representations:
$$\begin{array}{rcl}
\mu_i &=& \sum_j a_{i\rightarrow j} (h_i^{(t)} - h_j^{(t)}) = h_i^{(t)} - \sum_j a_{i\rightarrow j} h_j^{(t)}, GAT: \mu_i &=& \sum_j a_{i\rightarrow j} Wh_i^{(t)}\\
\mu_j &=& \sum_i a_{j\rightarrow i} (h_j^{(t)} - h_i^{(t)}) = h_j^{(t)} - \sum_i a_{j\rightarrow i} h_i^{(t)}.
\end{array}
$$
$$
h_i^{(t+1)} = f_\mathrm{node}\left(h_i^{(t)}, \sum_{j:(j,i)\in E} m_{j\rightarrow i}, \mu_i\right). GAT: h_i^{(t+1)} = \sigma \left(\mu_i\right)
$$
3) aggregate node representations to get graph representations

$$h_G = \mathrm{MLP_G}\left(\sum_{i\in V} h_i^{(T)}\right).$$
$$h_G = \mathrm{MLP_G}\left(\sum_{i\in V} \sigma(\mathrm{MLP_{gate}}(h_i^{(T)})) \odot \mathrm{MLP}(h_i^{(T)})\right).$$

2. $d_H$ is an existing distance metric\:

Euclidean distance: 

$d_H(x, y) = \sqrt{\sum_{i=1}^H (x_i - y_i)^2}$

or Hamming distance: 

$d_H(x, y)=\sum_{i=1}^H \mathbb{I}[x_i \ne y_i]$

or dot product:

or cosine:


In [ ]:
def pairwise_euclidean_similarity(x, y):
    """Compute the pairwise Euclidean similarity between x and y.
    This function computes the following similarity value between each pair of x_i
    and y_j: s(x_i, y_j) = -|x_i - y_j|^2.
    Args:
      x: NxD float tensor.
      y: MxD float tensor.
    Returns:
      s: NxM float tensor, the pairwise euclidean similarity.
    """
    s = 2 * torch.mm(x, torch.transpose(y, 1, 0))
    diag_x = torch.sum(x * x, dim=-1)
    diag_x = torch.unsqueeze(diag_x, 0)
    diag_y = torch.reshape(torch.sum(y * y, dim=-1), (1, -1))

    return s - diag_x - diag_y


def pairwise_dot_product_similarity(x, y):
    """Compute the dot product similarity between x and y.
    This function computes the following similarity value between each pair of x_i
    and y_j: s(x_i, y_j) = x_i^T y_j.
    Args:
      x: NxD float tensor.
      y: MxD float tensor.
    Returns:
      s: NxM float tensor, the pairwise dot product similarity.
    """
    return torch.mm(x, torch.transpose(y, 1, 0))


def pairwise_cosine_similarity(x, y):
    """Compute the cosine similarity between x and y.
    This function computes the following similarity value between each pair of x_i
    and y_j: s(x_i, y_j) = x_i^T y_j / (|x_i||y_j|).
    Args:
      x: NxD float tensor.
      y: MxD float tensor.
    Returns:
      s: NxM float tensor, the pairwise cosine similarity.
    """
    x = torch.div(x, torch.sqrt(torch.max(torch.sum(x ** 2), 1e-12)))
    y = torch.div(y, torch.sqrt(torch.max(torch.sum(y ** 2), 1e-12)))
    return torch.mm(x, torch.transpose(y, 1, 0))


PAIRWISE_SIMILARITY_FUNCTION = {
    'euclidean': pairwise_euclidean_similarity,
    'dotproduct': pairwise_dot_product_similarity,
    'cosine': pairwise_cosine_similarity,
}


def get_pairwise_similarity(name):
    """Get pairwise similarity metric by name.
    Args:
      name: string, name of the similarity metric, one of {dot-product, cosine,
        euclidean}.
    Returns:
      similarity: a (x, y) -> sim function.
    Raises:
      ValueError: if name is not supported.
    """
    if name not in PAIRWISE_SIMILARITY_FUNCTION:
        raise ValueError('Similarity metric name "%s" not supported.' % name)
    else:
        return PAIRWISE_SIMILARITY_FUNCTION[name]

#### Cross-graph attention

In [ ]:
def compute_cross_attention(x, y, sim):
    """Compute cross attention.
    x_i attend to y_j:
    a_{i->j} = exp(sim(x_i, y_j)) / sum_j exp(sim(x_i, y_j))
    y_j attend to x_i:
    a_{j->i} = exp(sim(x_i, y_j)) / sum_i exp(sim(x_i, y_j))
    attention_x = sum_j a_{i->j} y_j
    attention_y = sum_i a_{j->i} x_i
    Args:
      x: NxD float tensor.
      y: MxD float tensor.
      sim: a (x, y) -> similarity function.
    Returns:
      attention_x: NxD float tensor.
      attention_y: NxD float tensor.
    """
    a = sim(x, y)
    a_x = torch.softmax(a, dim=1)  # i->j
    a_y = torch.softmax(a, dim=0)  # j->i
    attention_x = torch.mm(a_x, y)
    attention_y = torch.mm(torch.transpose(a_y, 1, 0), x)
    return attention_x, attention_y





In [ ]:
def batch_block_pair_attention(data,
                               block_idx,
                               n_blocks,
                               similarity='dotproduct'):
    """Compute batched attention between pairs of blocks.
    This function partitions the batch data into blocks according to block_idx.
    For each pair of blocks, x = data[block_idx == 2i], and
    y = data[block_idx == 2i+1], we compute
    x_i attend to y_j:
    a_{i->j} = exp(sim(x_i, y_j)) / sum_j exp(sim(x_i, y_j))
    y_j attend to x_i:
    a_{j->i} = exp(sim(x_i, y_j)) / sum_i exp(sim(x_i, y_j))
    and
    attention_x = sum_j a_{i->j} y_j
    attention_y = sum_i a_{j->i} x_i.
    Args:
      data: NxD float tensor.
      block_idx: N-dim int tensor.
      n_blocks: integer.
      similarity: a string, the similarity metric.
    Returns:
      attention_output: NxD float tensor, each x_i replaced by attention_x_i.
    Raises:
      ValueError: if n_blocks is not an integer or not a multiple of 2.
    """
    if not isinstance(n_blocks, int):
        raise ValueError('n_blocks (%s) has to be an integer.' % str(n_blocks))

    if n_blocks % 2 != 0:
        raise ValueError('n_blocks (%d) must be a multiple of 2.' % n_blocks)

    sim = get_pairwise_similarity(similarity)

    results = []

    # This is probably better than doing boolean_mask for each i
    partitions = []
    for i in range(n_blocks):
        partitions.append(data[block_idx == i, :])

    for i in range(0, n_blocks, 2):
        x = partitions[i]
        y = partitions[i + 1]
        attention_x, attention_y = compute_cross_attention(x, y, sim)
        results.append(attention_x)
        results.append(attention_y)
    results = torch.cat(results, dim=0)

    return results

#### Graph matching layer and graph matching networks

In [ ]:
class GraphPropMatchingLayer(GraphPropLayer):
    """A graph propagation layer that also does cross graph matching.
    It assumes the incoming graph data is batched and paired, i.e. graph 0 and 1
    forms the first pair and graph 2 and 3 are the second pair etc., and computes
    cross-graph attention-based matching for each pair.
    """

    def forward(self,
                node_states,
                from_idx,
                to_idx,
                graph_idx,
                n_graphs,
                similarity='dotproduct',
                edge_features=None,
                node_features=None):
        """Run one propagation step with cross-graph matching.
        Args:
          node_states: [n_nodes, node_state_dim] float tensor, node states.
          from_idx: [n_edges] int tensor, from node indices for each edge.
          to_idx: [n_edges] int tensor, to node indices for each edge.
          graph_idx: [n_onodes] int tensor, graph id for each node.
          n_graphs: integer, number of graphs in the batch.
          similarity: type of similarity to use for the cross graph attention.
          edge_features: if not None, should be [n_edges, edge_feat_dim] tensor,
            extra edge features.
          node_features: if not None, should be [n_nodes, node_feat_dim] tensor,
            extra node features.
        Returns:
          node_states: [n_nodes, node_state_dim] float tensor, new node states.
        Raises:
          ValueError: if some options are not provided correctly.
        """
        aggregated_messages = self._compute_aggregated_messages(
            node_states, from_idx, to_idx, edge_features=edge_features)

        cross_graph_attention = batch_block_pair_attention(
            node_states, graph_idx, n_graphs, similarity=similarity)
        attention_input = node_states - cross_graph_attention

        return self._compute_node_update(node_states,
                                         [aggregated_messages, attention_input],
                                         node_features=node_features)




In [ ]:
class GraphMatchingNet(GraphEmbeddingNet):
    """Graph matching net.
    This class uses graph matching layers instead of the simple graph prop layers.
    It assumes the incoming graph data is batched and paired, i.e. graph 0 and 1
    forms the first pair and graph 2 and 3 are the second pair etc., and computes
    cross-graph attention-based matching for each pair.
    """

    def __init__(self,
                 encoder,
                 aggregator,
                 node_state_dim,
                 edge_state_dim,
                 edge_hidden_sizes,
                 node_hidden_sizes,
                 n_prop_layers,
                 share_prop_params=False,
                 edge_net_init_scale=0.1,
                 node_update_type='residual',
                 use_reverse_direction=True,
                 reverse_dir_param_different=True,
                 layer_norm=False,
                 layer_class=GraphPropLayer,
                 similarity='dotproduct',
                 prop_type='embedding'):
        super(GraphMatchingNet, self).__init__(
            encoder,
            aggregator,
            node_state_dim,
            edge_state_dim,
            edge_hidden_sizes,
            node_hidden_sizes,
            n_prop_layers,
            share_prop_params=share_prop_params,
            edge_net_init_scale=edge_net_init_scale,
            node_update_type=node_update_type,
            use_reverse_direction=use_reverse_direction,
            reverse_dir_param_different=reverse_dir_param_different,
            layer_norm=layer_norm,
            layer_class=GraphPropMatchingLayer,
            prop_type=prop_type,
        )
        self._similarity = similarity

    def _apply_layer(self,
                     layer,
                     node_states,
                     from_idx,
                     to_idx,
                     graph_idx,
                     n_graphs,
                     edge_features):
        """Apply one layer on the given inputs."""
        return layer(node_states, from_idx, to_idx, graph_idx, n_graphs,
                     similarity=self._similarity, edge_features=edge_features)

## Training

### Training on pairs (loss)

Euclidean distance: $d_H(x, y) = \sqrt{\sum_{i=1}^H (x_i - y_i)^2}$,

or Hamming distance: $d_H(x, y)=\sum_{i=1}^H \mathbb{I}[x_i \ne y_i]$

In [ ]:
def euclidean_distance(x, y):
    """This is the squared Euclidean distance."""
    return torch.sum((x - y) ** 2, dim=-1)


def approximate_hamming_similarity(x, y):
    """Approximate Hamming similarity."""
    return torch.mean(torch.tanh(x) * torch.tanh(y), dim=1)


def pairwise_loss(x, y, labels, loss_type='margin', margin=1.0):
    """Compute pairwise loss.
    Args:
      x: [N, D] float tensor, representations for N examples.
      y: [N, D] float tensor, representations for another N examples.
      labels: [N] int tensor, with values in -1 or +1.  labels[i] = +1 if x[i]
        and y[i] are similar, and -1 otherwise.
      loss_type: margin or hamming.
      margin: float scalar, margin for the margin loss.
    Returns:
      loss: [N] float tensor.  Loss for each pair of representations.
    """

    labels = labels.float()

    if loss_type == 'margin':
        return torch.relu(margin - labels * (1 - euclidean_distance(x, y)))
    elif loss_type == 'hamming':
        return 0.25 * (labels - approximate_hamming_similarity(x, y)) ** 2
    else:
        raise ValueError('Unknown loss_type %s' % loss_type)


### Training on triplets (loss)

margin, hamming

In [ ]:
def triplet_loss(x_1, y, x_2, z, loss_type='margin', margin=1.0):
    """Compute triplet loss.
    This function computes loss on a triplet of inputs (x, y, z).  A similarity or
    distance value is computed for each pair of (x, y) and (x, z).  Since the
    representations for x can be different in the two pairs (like our matching
    model) we distinguish the two x representations by x_1 and x_2.
    Args:
      x_1: [N, D] float tensor.
      y: [N, D] float tensor.
      x_2: [N, D] float tensor.
      z: [N, D] float tensor.
      loss_type: margin or hamming.
      margin: float scalar, margin for the margin loss.
    Returns:
      loss: [N] float tensor.  Loss for each pair of representations.
    """
    if loss_type == 'margin':
        return torch.relu(margin +
                          euclidean_distance(x_1, y) -
                          euclidean_distance(x_2, z))
    elif loss_type == 'hamming':
        return 0.125 * ((approximate_hamming_similarity(x_1, y) - 1) ** 2 +
                        (approximate_hamming_similarity(x_2, z) + 1) ** 2)
    else:
        raise ValueError('Unknown loss_type %s' % loss_type)

## Datasets


In [ ]:
import abc
import collections

"""A general Interface"""

GraphData = collections.namedtuple('GraphData', [
    'from_idx',
    'to_idx',
    'node_features',
    'edge_features',
    'graph_idx',
    'n_graphs'])

class GraphSimilarityDataset(object):
    """Base class for all the graph similarity learning datasets.
  This class defines some common interfaces a graph similarity dataset can have,
  in particular the functions that creates iterators over pairs and triplets.
  """

    @abc.abstractmethod
    def triplets(self, batch_size):
        """Create an iterator over triplets.
    Args:
      batch_size: int, number of triplets in a batch.
    Yields:
      graphs: a `GraphData` instance.  The batch of triplets put together.  Each
        triplet has 3 graphs (x, y, z).  Here the first graph is duplicated once
        so the graphs for each triplet are ordered as (x, y, x, z) in the batch.
        The batch contains `batch_size` number of triplets, hence `4*batch_size`
        many graphs.
    """
        pass

    @abc.abstractmethod
    def pairs(self, batch_size):
        """Create an iterator over pairs.
    Args:
      batch_size: int, number of pairs in a batch.
    Yields:
      graphs: a `GraphData` instance.  The batch of pairs put together.  Each
        pair has 2 graphs (x, y).  The batch contains `batch_size` number of
        pairs, hence `2*batch_size` many graphs.
      labels: [batch_size] int labels for each pair, +1 for similar, -1 for not.
    """
        pass


## Graph edit distance task

### Graph manipulation primitives


In [ ]:
import networkx as nx

"""Graph Edit Distance Task"""


# Graph Manipulation Functions
def permute_graph_nodes(g):
    """Permute node ordering of a graph, returns a new graph."""
    n = g.number_of_nodes()
    new_g = nx.Graph()
    new_g.add_nodes_from(range(n))
    perm = np.random.permutation(n)
    edges = g.edges()
    new_edges = []
    for x, y in edges:
        new_edges.append((perm[x], perm[y]))
    new_g.add_edges_from(new_edges)
    return new_g


def substitute_random_edges(g, n):
    """Substitutes n edges from graph g with another n randomly picked edges."""
    g = copy.deepcopy(g)
    n_nodes = g.number_of_nodes()
    edges = list(g.edges())
    # sample n edges without replacement
    e_remove = [
        edges[i] for i in np.random.choice(np.arange(len(edges)), n, replace=False)
    ]
    edge_set = set(edges)
    e_add = set()
    while len(e_add) < n:
        e = np.random.choice(n_nodes, 2, replace=False)
        # make sure e does not exist and is not already chosen to be added
        if (
                (e[0], e[1]) not in edge_set
                and (e[1], e[0]) not in edge_set
                and (e[0], e[1]) not in e_add
                and (e[1], e[0]) not in e_add
        ):
            e_add.add((e[0], e[1]))

    for i, j in e_remove:
        g.remove_edge(i, j)
    for i, j in e_add:
        g.add_edge(i, j)
    return g

### Dataset for training, fixed dataset for evaluation


In [ ]:
import contextlib
import networkx as nx
import numpy as np


class GraphEditDistanceDataset(GraphSimilarityDataset):
    """Graph edit distance dataset."""

    def __init__(
            self,
            n_nodes_range,
            p_edge_range,
            n_changes_positive,
            n_changes_negative,
            permute=True,
    ):
        """Constructor.
    Args:
      n_nodes_range: a tuple (n_min, n_max).  The minimum and maximum number of
        nodes in a graph to generate.
      p_edge_range: a tuple (p_min, p_max).  The minimum and maximum edge
        probability.
      n_changes_positive: the number of edge substitutions for a pair to be
        considered positive (similar).
      n_changes_negative: the number of edge substitutions for a pair to be
        considered negative (not similar).
      permute: if True (default), permute node orderings in addition to
        changing edges; if False, the node orderings across a pair or triplet of
        graphs will be the same, useful for visualization.
    """
        self._n_min, self._n_max = n_nodes_range
        self._p_min, self._p_max = p_edge_range
        self._k_pos = n_changes_positive
        self._k_neg = n_changes_negative
        self._permute = permute

    def _get_graph(self):
        """Generate one graph."""
        n_nodes = np.random.randint(self._n_min, self._n_max + 1)
        p_edge = np.random.uniform(self._p_min, self._p_max)

        # do a little bit of filtering
        n_trials = 100
        for _ in range(n_trials):
            g = nx.erdos_renyi_graph(n_nodes, p_edge)
            if nx.is_connected(g):
                return g

        raise ValueError("Failed to generate a connected graph.")

    def _get_pair(self, positive):
        """Generate one pair of graphs."""
        g = self._get_graph()
        if self._permute:
            permuted_g = permute_graph_nodes(g)
        else:
            permuted_g = g
        n_changes = self._k_pos if positive else self._k_neg
        changed_g = substitute_random_edges(g, n_changes)
        return permuted_g, changed_g

    def _get_triplet(self):
        """Generate one triplet of graphs."""
        g = self._get_graph()
        if self._permute:
            permuted_g = permute_graph_nodes(g)
        else:
            permuted_g = g
        pos_g = substitute_random_edges(g, self._k_pos)
        neg_g = substitute_random_edges(g, self._k_neg)
        return permuted_g, pos_g, neg_g

    def triplets(self, batch_size):
        """Yields batches of triplet data."""
        while True:
            batch_graphs = []
            for _ in range(batch_size):
                g1, g2, g3 = self._get_triplet()
                batch_graphs.append((g1, g2, g1, g3))
            yield self._pack_batch(batch_graphs)

    def pairs(self, batch_size):
        """Yields batches of pair data."""
        while True:
            batch_graphs = []
            batch_labels = []
            positive = True
            for _ in range(batch_size):
                g1, g2 = self._get_pair(positive)
                batch_graphs.append((g1, g2))
                batch_labels.append(1 if positive else -1)
                positive = not positive

            packed_graphs = self._pack_batch(batch_graphs)
            labels = np.array(batch_labels, dtype=np.int32)
            yield packed_graphs, labels

    def _pack_batch(self, graphs):
        """Pack a batch of graphs into a single `GraphData` instance.
    Args:
      graphs: a list of generated networkx graphs.
    Returns:
      graph_data: a `GraphData` instance, with node and edge indices properly
        shifted.
    """
        Graphs = []
        for graph in graphs:
            for inergraph in graph:
                Graphs.append(inergraph)
        graphs = Graphs
        from_idx = []
        to_idx = []
        graph_idx = []

        n_total_nodes = 0
        n_total_edges = 0
        for i, g in enumerate(graphs):
            n_nodes = g.number_of_nodes()
            n_edges = g.number_of_edges()
            edges = np.array(g.edges(), dtype=np.int32)
            # shift the node indices for the edges
            from_idx.append(edges[:, 0] + n_total_nodes)
            to_idx.append(edges[:, 1] + n_total_nodes)
            graph_idx.append(np.ones(n_nodes, dtype=np.int32) * i)

            n_total_nodes += n_nodes
            n_total_edges += n_edges

        GraphData = collections.namedtuple('GraphData', [
            'from_idx',
            'to_idx',
            'node_features',
            'edge_features',
            'graph_idx',
            'n_graphs'])

        return GraphData(
            from_idx=np.concatenate(from_idx, axis=0),
            to_idx=np.concatenate(to_idx, axis=0),
            # this task only cares about the structures, the graphs have no features.
            # setting higher dimension of ones to confirm code functioning
            # with high dimensional features.
            node_features=np.ones((n_total_nodes, 8), dtype=np.float32),
            edge_features=np.ones((n_total_edges, 4), dtype=np.float32),
            graph_idx=np.concatenate(graph_idx, axis=0),
            n_graphs=len(graphs),
        )


In [ ]:


# Use Fixed datasets for evaluation
@contextlib.contextmanager
def reset_random_state(seed):
    """This function creates a context that uses the given seed."""
    np_rnd_state = np.random.get_state()
    rnd_state = random.getstate()
    np.random.seed(seed)
    random.seed(seed + 1)
    try:
        yield
    finally:
        random.setstate(rnd_state)
        np.random.set_state(np_rnd_state)


In [ ]:


class FixedGraphEditDistanceDataset(GraphEditDistanceDataset):
    """A fixed dataset of pairs or triplets for the graph edit distance task.
  This dataset can be used for evaluation.
  """

    def __init__(
            self,
            n_nodes_range,
            p_edge_range,
            n_changes_positive,
            n_changes_negative,
            dataset_size,
            permute=True,
            seed=1234,
    ):
        super(FixedGraphEditDistanceDataset, self).__init__(
            n_nodes_range,
            p_edge_range,
            n_changes_positive,
            n_changes_negative,
            permute=permute,
        )
        self._dataset_size = dataset_size
        self._seed = seed

    def triplets(self, batch_size):
        """Yield triplets."""

        if hasattr(self, "_triplets"):
            triplets = self._triplets
        else:
            # get a fixed set of triplets
            with reset_random_state(self._seed):
                triplets = []
                for _ in range(self._dataset_size):
                    g1, g2, g3 = self._get_triplet()
                    triplets.append((g1, g2, g1, g3))
            self._triplets = triplets

        ptr = 0
        while ptr + batch_size <= len(triplets):
            batch_graphs = triplets[ptr: ptr + batch_size]
            yield self._pack_batch(batch_graphs)
            ptr += batch_size

    def pairs(self, batch_size):
        """Yield pairs and labels."""

        if hasattr(self, "_pairs") and hasattr(self, "_labels"):
            pairs = self._pairs
            labels = self._labels
        else:
            # get a fixed set of pairs first
            with reset_random_state(self._seed):
                pairs = []
                labels = []
                positive = True
                for _ in range(self._dataset_size):
                    pairs.append(self._get_pair(positive))
                    labels.append(1 if positive else -1)
                    positive = not positive
            labels = np.array(labels, dtype=np.int32)

            self._pairs = pairs
            self._labels = labels

        ptr = 0
        while ptr + batch_size <= len(pairs):
            batch_graphs = pairs[ptr: ptr + batch_size]
            packed_batch = self._pack_batch(batch_graphs)
            yield packed_batch, labels[ptr: ptr + batch_size]
            ptr += batch_size

###QM7b

In [ ]:
!pip install dgl -q
from dgl.data import QM7bDataset
qm = QM7bDataset()


class QM7b():
  #只截取前20个图  177 个节点	 1713 个边 生成的微调图会删去两条边 3386个边 删去的边而影响的节点？？？？354个节点
  def pairs(self,batch_size):#读前20个图微调生成另外20个，打包成namedtuple
    while True:
      batch_graphs = []
      batch_labels = []
      positive = True 
      for i in range(batch_size):#默认20
        g1,_ = qm[i]
        g2 = g1.clone()
        if positive:#True  移除2个from_edge_idx
          remove_idx = np.random.choice(np.arange(g2.num_edges()),1,replace=False)
        else:#False 移除3个from_edge_idx
          remove_idx = np.random.choice(np.arange(g2.num_edges()),2,replace=False)
        g2.remove_edges(remove_idx)
        batch_graphs.append((g1, g2))#list里添20个tuple
        batch_labels.append(1 if positive else -1)#list里20个整数
        positive = not positive#一正一负
      labels = np.array(batch_labels, dtype=np.int32)
      from_idx = []
      to_idx = []
      graph_idx = []
      graphs = []
      num_nodes = 0
      num_edges = 0
      for tuples in batch_graphs:#拆list 20个元祖
        for pair in tuples:#拆元组 2个图
          graphs.append(pair)#list里40个图
      batch_graphs = graphs
      for i,g in enumerate(batch_graphs):#返回的是索引以及索引对应元素
        edge_idx = torch.arange(0,g.num_edges())
        src, dst = g.find_edges(edge_idx)
        src = np.array(src, dtype=np.int32)
        dst = np.array(dst, dtype=np.int32)
        from_idx.append(src)
        to_idx.append(dst)
        num_nodes += g.num_nodes()
        num_edges += g.num_edges()
        graph_idx.append(np.ones(g.num_nodes(), dtype=np.int32) * i)
      GraphData = collections.namedtuple('GraphData', ['from_idx','to_idx','node_features','edge_features', 'graph_idx','n_graphs'])
      packed_graphs = GraphData(
              from_idx=np.concatenate(from_idx, axis=0),
              to_idx=np.concatenate(to_idx, axis=0),
              node_features=np.ones((num_nodes, 8), dtype=np.float32),
              edge_features=np.ones((num_edges, 4), dtype=np.float32),
              graph_idx=np.concatenate(graph_idx, axis=0),
              n_graphs=len(batch_graphs),
      )
      yield packed_graphs, labels

     |████████████████████████████████| 4.4 MB 4.3 MB/s 
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [ ]:
class QM7bVali():
  def pairs(self,batch_size):
    batch_graphs = []
    batch_labels = []
    positive = True 
    for i in range(1000):#默认20还是1000
      g1,_ = qm[i+20]
      g2 = g1.clone()
      if positive:#True  移除2个from_edge_idx
        remove_idx =  np.random.choice(np.arange(g2.num_edges()),1,replace=False)
      else:#False 移除3个from_edge_idx
        remove_idx = np.random.choice(np.arange(g2.num_edges()),5,replace=False)
      g2.remove_edges(remove_idx)
      batch_graphs.append((g1, g2))#list里添20个tuple
      batch_labels.append(1 if positive else -1)#list里20个整数
      positive = not positive#一正一负
    labels = np.array(batch_labels, dtype=np.int32)

    ptr = 0
    while ptr + batch_size <= len(batch_graphs):
      batch_graphs = batch_graphs[ptr: ptr + batch_size]
      from_idx = []
      to_idx = []
      graph_idx = []
      graphs = []
      num_nodes = 0
      num_edges = 0
      for tuples in batch_graphs:#拆list 20个元祖
        for pair in tuples:#拆元组 2个图
          graphs.append(pair)#list里40个图
      for i,g in enumerate(graphs):#返回的是索引以及索引对应元素
        edge_idx = torch.arange(0,g.num_edges())
        src, dst = g.find_edges(edge_idx)
        src = np.array(src, dtype=np.int32)
        dst = np.array(dst, dtype=np.int32)
        from_idx.append(src)
        to_idx.append(dst)
        num_nodes += g.num_nodes()
        num_edges += g.num_edges()
        graph_idx.append(np.ones(g.num_nodes(), dtype=np.int32) * i)
      GraphData = collections.namedtuple('GraphData', ['from_idx','to_idx','node_features','edge_features', 'graph_idx','n_graphs'])
      packed_graphs = GraphData(
              from_idx=np.concatenate(from_idx, axis=0),
              to_idx=np.concatenate(to_idx, axis=0),
              node_features=np.ones((num_nodes, 8), dtype=np.float32),
              edge_features=np.ones((num_edges, 4), dtype=np.float32),
              graph_idx=np.concatenate(graph_idx, axis=0),
              n_graphs=len(graphs),
      )
      yield packed_graphs, labels[ptr: ptr + batch_size]
      ptr += batch_size

  def triplets(self, batch_size):
        
    triplets = []
    for i in range(1000):
        g1,_ = qm[i+20]
        g2 = g1.clone()
        g3 = g1.clone()
        remove_idx =  np.random.choice(np.arange(g2.num_edges()),1,replace=False)
        g2.remove_edges(remove_idx)
        remove_idx =  np.random.choice(np.arange(g2.num_edges()),5,replace=False)
        g3.remove_edges(remove_idx)
        triplets.append((g1, g2, g1, g3))

    ptr = 0
    while ptr + batch_size <= len(triplets):
      batch_graphs = triplets[ptr: ptr + batch_size]

      from_idx = []
      to_idx = []
      graph_idx = []
      graphs = []
      num_nodes = 0
      num_edges = 0
      for tuples in batch_graphs:#拆list 20个元祖
        for pair in tuples:#拆元组 2个图
          graphs.append(pair)#list里40个图
      for i,g in enumerate(graphs):#返回的是索引以及索引对应元素
        edge_idx = torch.arange(0,g.num_edges())
        src, dst = g.find_edges(edge_idx)
        src = np.array(src, dtype=np.int32)
        dst = np.array(dst, dtype=np.int32)
        from_idx.append(src)
        to_idx.append(dst)
        num_nodes += g.num_nodes()
        num_edges += g.num_edges()
        graph_idx.append(np.ones(g.num_nodes(), dtype=np.int32) * i)
      GraphData = collections.namedtuple('GraphData', ['from_idx','to_idx','node_features','edge_features', 'graph_idx','n_graphs'])
      packed_graphs = GraphData(
              from_idx=np.concatenate(from_idx, axis=0),
              to_idx=np.concatenate(to_idx, axis=0),
              node_features=np.ones((num_nodes, 8), dtype=np.float32),
              edge_features=np.ones((num_edges, 4), dtype=np.float32),
              graph_idx=np.concatenate(graph_idx, axis=0),
              n_graphs=len(graphs),
      )
      yield packed_graphs
      ptr += batch_size
      

## Building the model, and the training and evaluation pipelines

### Configs


In [ ]:
def get_default_config():
    """The default configs."""
    model_type = 'matching'  #如果用嵌入网络就设置为 `embedding
    # Set to `embedding` to use the graph embedding net.
    node_state_dim = 32
    edge_state_dim = 16
    graph_rep_dim = 128
    graph_embedding_net_config = dict(
        node_state_dim=node_state_dim,
        edge_state_dim=edge_state_dim,
        edge_hidden_sizes=[node_state_dim * 2, node_state_dim * 2],
        node_hidden_sizes=[node_state_dim * 2],
        n_prop_layers=5,
        # set to False to not share parameters across message passing layers
        share_prop_params=True,   # 判断在信息传递层是否参数共享
        # initialize message MLP with small parameter weights to prevent
        # aggregated message vectors blowing up, alternatively we could also use
        # e.g. layer normalization to keep the scale of these under control.
        edge_net_init_scale=0.1,#用小参数权重初始化消息 MLP 以防止聚合消息向量爆炸
	                              #或者也可以使用例如 层标准化以控制这些的规模。
        # other types of update like `mlp` and `residual` can also be used here. gru
        node_update_type='gru', # 其他也可以用'mlp'  `residual`
        # set to False if your graph already contains edges in both directions.
        use_reverse_direction=True,     #如果有双向边就设成FALSE
        # set to True if your graph is directed
        reverse_dir_param_different=False,  #如果是有向图则设成TRUE
        # we didn't use layer norm in our experiments but sometimes this can help.
        layer_norm=False,# 在实验中没有使用层范数，但有时会有用
        # set to `embedding` to use the graph embedding net.
        prop_type=model_type)
    graph_matching_net_config = graph_embedding_net_config.copy()
    graph_matching_net_config['similarity'] = 'dotproduct'  # other: euclidean, cosine
    return dict(
        encoder=dict(
            node_hidden_sizes=[node_state_dim],
            node_feature_dim=1,
            edge_hidden_sizes=[edge_state_dim]),
        aggregator=dict(
            node_hidden_sizes=[graph_rep_dim],
            graph_transform_sizes=[graph_rep_dim],
            input_size=[node_state_dim],
            gated=True,
            aggregation_type='sum'),
        graph_embedding_net=graph_embedding_net_config,
        graph_matching_net=graph_matching_net_config,
        model_type=model_type,
        data=dict(
            problem='graph_edit_distance',
            dataset_params=dict(
                # always generate graphs with 20 nodes and p_edge=0.2.
                n_nodes_range=[20, 20],
                p_edge_range=[0.2, 0.2],#生成具有 20 个节点且 p_edge=0.2 （边概率？）的图
                n_changes_positive=1,  #一对被视为正（相似）的边替换的数量
                n_changes_negative=2,
                validation_dataset_size=1000)),
        training=dict(
            batch_size=20,
            learning_rate=1e-4,
            mode='pair',
            loss='margin',  # other: hamming
            margin=1.0,
            # A small regularizer on the graph vector scales to avoid the graph
            # vectors blowing up.  If numerical issues is particularly bad in the
            # model we can add `snt.LayerNorm` to the outputs of each layer, the
            # aggregated messages and aggregated node representations to
            # keep the network activation scale in a reasonable range.
            graph_vec_regularizer_weight=1e-6,	  #图向量上有一个小的正则化器会缩放以避免图向量爆炸。 
                                                  #如果模型中的数值问题特别严重，
                                                  #可以将 `snt.LayerNorm` 添加到每一层的输出、聚合消息和聚合节点表示中，
                                                  #以将网络激活规模保持在合理范围内。
            # Add gradient clipping to avoid large gradients.
            clip_value=10.0,   #设置梯度裁剪防止梯度爆炸
            # '''
            # 梯度爆炸：在神经网络中，当前面隐藏层的学习速率低于后面隐藏层的学习速率，
            # 即随着隐藏层数目的增加，分类准确率反而下降了。 这种现象叫梯度爆炸。 
            # 其实梯度消失和梯度爆炸是一回事，只是表现的形式，以及产生的原因不一样。
            # '''
            # Increase this to train longer.
            n_training_steps=500000,   #控制训练时长
            # Print training information every this many training steps.
            print_after=100,   #每隔这么多训练步骤打印训练信息
            # Evaluate on validation set every `eval_after * print_after` steps.
            eval_after=10),    #每个 `eval_after * print_after` 步骤对验证集进行评估。
        evaluation=dict(
            batch_size=20),
        seed=8,
    )

### Evaluation


In [ ]:
from sklearn import metrics

def exact_hamming_similarity(x, y):
    """Compute the binary Hamming similarity."""
    match = ((x > 0) * (y > 0)).float()
    return torch.mean(match, dim=1)


def compute_similarity(config, x, y):
    """Compute the distance between x and y vectors.
    The distance will be computed based on the training loss type.
    Args:
      config: a config dict.
      x: [n_examples, feature_dim] float tensor.
      y: [n_examples, feature_dim] float tensor.
    Returns:
      dist: [n_examples] float tensor.
    Raises:
      ValueError: if loss type is not supported.
    """
    if config['training']['loss'] == 'margin':
        # similarity is negative distance
        return -euclidean_distance(x, y)
    elif config['training']['loss'] == 'hamming':
        return exact_hamming_similarity(x, y)
    else:
        raise ValueError('Unknown loss type %s' % config['training']['loss'])


In [ ]:
def auc(scores, labels, **auc_args):
    """Compute the AUC for pair classification.
    See `tf.metrics.auc` for more details about this metric.
    Args:
      scores: [n_examples] float.  Higher scores mean higher preference of being
        assigned the label of +1.
      labels: [n_examples] int.  Labels are either +1 or -1.
      **auc_args: other arguments that can be used by `tf.metrics.auc`.
    Returns:
      auc: the area under the ROC curve.
    """
    scores_max = torch.max(scores)
    scores_min = torch.min(scores)

    # normalize scores to [0, 1] and add a small epislon for safety
    scores = (scores - scores_min) / (scores_max - scores_min + 1e-8)

    labels = (labels + 1) / 2

    fpr, tpr, thresholds = metrics.roc_curve(labels.cpu().detach().numpy(), scores.cpu().detach().numpy())
    return metrics.auc(fpr, tpr)

### Build the model

In [ ]:
def reshape_and_split_tensor(tensor, n_splits):
    """Reshape and split a 2D tensor along the last dimension.
    Args:
      tensor: a [num_examples, feature_dim] tensor.  num_examples must be a
        multiple of `n_splits`.
      n_splits: int, number of splits to split the tensor into.
    Returns:
      splits: a list of `n_splits` tensors.  The first split is [tensor[0],
        tensor[n_splits], tensor[n_splits * 2], ...], the second split is
        [tensor[1], tensor[n_splits + 1], tensor[n_splits * 2 + 1], ...], etc..
    """
    feature_dim = tensor.shape[-1]
    tensor = torch.reshape(tensor, [-1, feature_dim * n_splits])
    tensor_split = []
    for i in range(n_splits):
        tensor_split.append(tensor[:, feature_dim * i: feature_dim * (i + 1)])
    return tensor_split


def build_model(config, node_feature_dim, edge_feature_dim):
    """Create model for training and evaluation.
    Args:
      config: a dictionary of configs, like the one created by the
        `get_default_config` function.
      node_feature_dim: int, dimensionality of node features.
      edge_feature_dim: int, dimensionality of edge features.
    Returns:
      tensors: a (potentially nested) name => tensor dict.
      placeholders: a (potentially nested) name => tensor dict.
      AE_model: a GraphEmbeddingNet or GraphMatchingNet instance.
    Raises:
      ValueError: if the specified model or training settings are not supported.
    """
    config['encoder']['node_feature_dim'] = node_feature_dim
    config['encoder']['edge_feature_dim'] = edge_feature_dim

    encoder = GraphEncoder(**config['encoder'])
    aggregator = GraphAggregator(**config['aggregator'])
    if config['model_type'] == 'embedding':
        model = GraphEmbeddingNet(
            encoder, aggregator, **config['graph_embedding_net'])
    elif config['model_type'] == 'matching':
        model = GraphMatchingNet(
            encoder, aggregator, **config['graph_matching_net'])
    else:
        raise ValueError('Unknown model type: %s' % config['model_type'])

    optimizer = torch.optim.Adam((model.parameters()),
                                 lr=config['training']['learning_rate'], weight_decay=1e-5)

    return model, optimizer




### build the dataset

In [ ]:
import copy

def build_datasets(config):
    """Build the training and evaluation datasets."""
    config = copy.deepcopy(config)

    if config['data']['problem'] == 'graph_edit_distance':
        dataset_params = config['data']['dataset_params']
        validation_dataset_size = dataset_params['validation_dataset_size']
        del dataset_params['validation_dataset_size']
        training_set = GraphEditDistanceDataset(**dataset_params)
        dataset_params['dataset_size'] = validation_dataset_size
        validation_set = FixedGraphEditDistanceDataset(**dataset_params)
    elif config['data']['problem'] == 'QM7b':
      training_set = QM7b()
      validation_set = QM7bVali()
    else:
        raise ValueError('Unknown problem type: %s' % config['data']['problem'])
    return training_set, validation_set


def get_graph(batch):
    if len(batch) != 2:
        # if isinstance(batch, GraphData):
        graph = batch
        node_features = torch.from_numpy(graph.node_features)
        edge_features = torch.from_numpy(graph.edge_features)
        from_idx = torch.from_numpy(graph.from_idx).long()
        to_idx = torch.from_numpy(graph.to_idx).long()
        graph_idx = torch.from_numpy(graph.graph_idx).long()
        return node_features, edge_features, from_idx, to_idx, graph_idx
    else:
        graph, labels = batch
        node_features = torch.from_numpy(graph.node_features)
        edge_features = torch.from_numpy(graph.edge_features)
        from_idx = torch.from_numpy(graph.from_idx).long()
        to_idx = torch.from_numpy(graph.to_idx).long()
        graph_idx = torch.from_numpy(graph.graph_idx).long()
        labels = torch.from_numpy(labels).long()
    return node_features, edge_features, from_idx, to_idx, graph_idx, labels

### Let's run it!

In [ ]:
import random
import time
import torch
import numpy as np

# Set GPU
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
#device = torch.device('cpu')


# Print configure
config = get_default_config()
for (k, v) in config.items():
    print("%s= %s" % (k, v))

# Set random seed
seed = config['seed']
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True


config['data']['problem'] = 'graph_edit_distance'
# config['data']['problem'] = 'QM7b'
config['model_type'] = 'matching'
# config['model_type'] = 'embedding'

training_set, validation_set = build_datasets(config)

if config['training']['mode'] == 'pair':
  #默认的是pair，默认的batch_size为20
    training_data_iter = training_set.pairs(config['training']['batch_size'])
    
    
    first_batch_graphs, _ = next(training_data_iter)
else:
    training_data_iter = training_set.triplets(config['training']['batch_size'])
    first_batch_graphs = next(training_data_iter)

node_feature_dim = first_batch_graphs.node_features.shape[-1]
edge_feature_dim = first_batch_graphs.edge_features.shape[-1]




model, optimizer = build_model(config, node_feature_dim, edge_feature_dim)
model.to(device)

accumulated_metrics = collections.defaultdict(list)


training_n_graphs_in_batch = config['training']['batch_size']
if config['training']['mode'] == 'pair':
    training_n_graphs_in_batch *= 2
elif config['training']['mode'] == 'triplet':
    training_n_graphs_in_batch *= 4
else:
    raise ValueError('Unknown training mode: %s' % config['training']['mode'])

t_start = time.time()
for i_iter in range(config['training']['n_training_steps']):
    model.train(mode=True)
    batch = next(training_data_iter)
    if config['training']['mode'] == 'pair':
        node_features, edge_features, from_idx, to_idx, graph_idx, labels = get_graph(batch)
        
        labels = labels.to(device)
    else:

        node_features, edge_features, from_idx, to_idx, graph_idx = get_graph(batch)

    graph_vectors = model(node_features.to(device), edge_features.to(device), from_idx.to(device), to_idx.to(device),graph_idx.to(device), training_n_graphs_in_batch)

    if config['training']['mode'] == 'pair':
        x, y = reshape_and_split_tensor(graph_vectors, 2)


        loss = pairwise_loss(x, y, labels,
                             loss_type=config['training']['loss'],
                             margin=config['training']['margin'])

        is_pos = (labels == torch.ones(labels.shape).long().to(device)).float()
        is_neg = 1 - is_pos
        n_pos = torch.sum(is_pos)
        n_neg = torch.sum(is_neg)



        sim = compute_similarity(config, x, y)
        sim_pos = torch.sum(sim * is_pos) / (n_pos + 1e-8)
        sim_neg = torch.sum(sim * is_neg) / (n_neg + 1e-8)
    else:
        x_1, y, x_2, z = reshape_and_split_tensor(graph_vectors, 4)
        loss = triplet_loss(x_1, y, x_2, z,
                            loss_type=config['training']['loss'],
                            margin=config['training']['margin'])

        sim_pos = torch.mean(compute_similarity(config, x_1, y))
        sim_neg = torch.mean(compute_similarity(config, x_2, z))

    graph_vec_scale = torch.mean(graph_vectors ** 2)
    if config['training']['graph_vec_regularizer_weight'] > 0:
        loss += (config['training']['graph_vec_regularizer_weight'] *
                 0.5 * graph_vec_scale)

    optimizer.zero_grad()


    loss.backward(torch.ones_like(loss))  #只支持pytorch1.2及以上
    nn.utils.clip_grad_value_(model.parameters(), config['training']['clip_value'])
    optimizer.step()

    sim_diff = sim_pos - sim_neg
    accumulated_metrics['loss'].append(loss)
    accumulated_metrics['sim_pos'].append(sim_pos)
    accumulated_metrics['sim_neg'].append(sim_neg)
    accumulated_metrics['sim_diff'].append(sim_diff)


    # evaluation
    if (i_iter + 1) % config['training']['print_after'] == 0:
        metrics_to_print = {
            k: torch.mean(v[0]) for k, v in accumulated_metrics.items()}
        info_str = ', '.join(
            ['%s %.4f' % (k, v) for k, v in metrics_to_print.items()])
        # reset the metrics
        accumulated_metrics = collections.defaultdict(list)

        if ((i_iter + 1) // config['training']['print_after'] %
                config['training']['eval_after'] == 0):
            model.eval()
            with torch.no_grad():
                accumulated_pair_auc = []
                for batch in validation_set.pairs(config['evaluation']['batch_size']):
                    node_features, edge_features, from_idx, to_idx, graph_idx, labels = get_graph(batch)
                    labels = labels.to(device)
                    eval_pairs = model(node_features.to(device), edge_features.to(device), from_idx.to(device),
                                       to_idx.to(device),
                                       graph_idx.to(device), config['evaluation']['batch_size'] * 2)

                    x, y = reshape_and_split_tensor(eval_pairs, 2)
                    similarity = compute_similarity(config, x, y)
                    pair_auc = auc(similarity, labels)
                    accumulated_pair_auc.append(pair_auc)

                accumulated_triplet_acc = []
                for batch in validation_set.triplets(config['evaluation']['batch_size']):
                    node_features, edge_features, from_idx, to_idx, graph_idx = get_graph(batch)
                    eval_triplets = model(node_features.to(device), edge_features.to(device), from_idx.to(device),
                                          to_idx.to(device),
                                          graph_idx.to(device),
                                          config['evaluation']['batch_size'] * 4)
                    x_1, y, x_2, z = reshape_and_split_tensor(eval_triplets, 4)
                    sim_1 = compute_similarity(config, x_1, y)
                    sim_2 = compute_similarity(config, x_2, z)
                    triplet_acc = torch.mean((sim_1 > sim_2).float())
                    accumulated_triplet_acc.append(triplet_acc.cpu().numpy())

                eval_metrics = {
                    'pair_auc': np.mean(accumulated_pair_auc),
                    'triplet_acc': np.mean(accumulated_triplet_acc)}
                info_str += ', ' + ', '.join(
                    ['%s %.4f' % ('val/' + k, v) for k, v in eval_metrics.items()])
            model.train()
        print('iter %d, %s, time %.2fs' % (
            i_iter + 1, info_str, time.time() - t_start))
        t_start = time.time()

encoder= {'node_hidden_sizes': [32], 'node_feature_dim': 1, 'edge_hidden_sizes': [16]}
aggregator= {'node_hidden_sizes': [128], 'graph_transform_sizes': [128], 'input_size': [32], 'gated': True, 'aggregation_type': 'sum'}
graph_embedding_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching'}
graph_matching_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching', 'similarity': 'dotproduct'}
model_type= matching
data= {'problem': 'graph_edit_distance', 'datase

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


iter 1000, loss 1.0307, sim_pos -1.0216, sim_neg -1.1651, sim_diff 0.1434, val/pair_auc 0.6844, val/triplet_acc 0.6910, time 26.29s
iter 1100, loss 0.9112, sim_pos -0.7195, sim_neg -0.9730, sim_diff 0.2536, time 16.63s
iter 1200, loss 0.7054, sim_pos -0.4615, sim_neg -1.2632, sim_diff 0.8017, time 16.60s
iter 1300, loss 1.0608, sim_pos -0.7628, sim_neg -0.6411, sim_diff -0.1217, time 16.73s
iter 1400, loss 0.8628, sim_pos -0.6238, sim_neg -0.9948, sim_diff 0.3711, time 16.89s
iter 1500, loss 0.6248, sim_pos -0.5409, sim_neg -1.3541, sim_diff 0.8132, time 16.51s
iter 1600, loss 0.7941, sim_pos -0.5874, sim_neg -1.1584, sim_diff 0.5710, time 16.56s
iter 1700, loss 0.7406, sim_pos -0.7468, sim_neg -1.5664, sim_diff 0.8196, time 16.53s
iter 1800, loss 0.7885, sim_pos -0.8307, sim_neg -1.7212, sim_diff 0.8905, time 16.82s
iter 1900, loss 0.6148, sim_pos -0.5820, sim_neg -1.7180, sim_diff 1.1360, time 16.44s
iter 2000, loss 0.8600, sim_pos -0.7455, sim_neg -1.4791, sim_diff 0.7336, val/pair_

# New Section